In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andradaolteanu/gtzan-dataset-music-genre-classification")

print("Path to dataset files:", path)

100%|██████████| 1.21G/1.21G [00:23<00:00, 56.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1


In [11]:
path = '/root/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1/Data'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import librosa
import librosa.display
import IPython.display as ipd

print(os.listdir(path))

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print("Number of available GPUs:", device_count)
    for i in range(device_count):
        print("GPU", i, ":", torch.cuda.get_device_name(i))
else:
    print("GPU is not available")

test_data = pd.read_csv(path + '/features_3_sec.csv')
test_data.head()

['features_3_sec.csv', 'features_30_sec.csv', 'genres_original', 'images_original']
GPU is not available


(9990, 60)

In [ ]:
#Test audio files
seed = 69
torch.manual_seed(seed)
np.random.seed(seed)
audio_path = path + '/Data/genres_original/'
classification_dict = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
hop_length = 512
for genre in classification_dict.keys():
    print(genre)

    # Reading the first audio file
    data,sampling_rate = librosa.load(audio_path + genre + "/" + genre + ".00000.wav")

    display(IPython.display.Audio(data, rate = sampling_rate))

In [ ]:
n_fft = 2048
n_mels = 128
output_genre_path = 'Mel_Spectrograms'
for genre in classification_dict.keys():
    if not os.path.exists(output_genre_path+"/"+genre):
        os.makedirs(output_genre_path+"/"+genre)
for genre in classification_dict.keys():
    for filename in os.listdir(audio_path + genre):
        if filename.endswith(".wav"):
            data, sampling_rate = librosa.load(audio_path + genre + "/" + filename)
            mel_spectrogram = librosa.feature.melspectrogram(y = data, sr = sampling_rate, n_fft = n_fft, hop_length = hop_length, n_mels = n_mels)
            mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Plot and save mel spectrogram
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(mel_spectrogram_db, sr=sampling_rate, x_axis='time', y_axis='mel', fmax=8000)
            plt.colorbar(format='%+2.0f dB')
            plt.title(f'Mel Spectrogram - {genre} - {filename}')
            plt.tight_layout()
            plt.savefig(os.path.join(output_genre_path+"/"+genre, f'{os.path.splitext(filename)[0]}.png'))
            plt.close()

print("Mel spectrograms saved by genre.")
